In [19]:
import pandas as pd
import nltk
from langdetect import detect, DetectorFactory
import swifter
from sklearn.utils import resample


nltk.download('words')
from nltk.corpus import words

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\kist0\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


<h3>데이터 불러오기</h3>

In [5]:
chunks = pd.read_csv("./user_track.csv", chunksize = 10000)
user_tracks = pd.DataFrame()
for chunk in chunks:
  user_tracks = pd.concat([user_tracks, chunk])


user_tracks

,track_uri,track_name,artist,user_id,playcount,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,lyrics
0,0BCPKOYdS2jbQ8iyB56Zns,Clocks,Coldplay,7241c0f8e15a4b9792ce5ddcfbe87a888f7f4b16,2,307879,0.577,0.749,5.0,-7.215,0.0,0.0279,0.599000,0.011200,0.1830,0.261,130.969,"The lights go out, and I can't be saved\n Tide..."
1,0BCPKOYdS2jbQ8iyB56Zns,Clocks,Coldplay,8b5838c03d615893231d98995ff57d3f7a6659b9,4,307879,0.577,0.749,5.0,-7.215,0.0,0.0279,0.599000,0.011200,0.1830,0.261,130.969,"The lights go out, and I can't be saved\n Tide..."
2,0BCPKOYdS2jbQ8iyB56Zns,Clocks,Coldplay,0fadffbb32e66100e27afc6a8a4a758ad8e9278b,1,307879,0.577,0.749,5.0,-7.215,0.0,0.0279,0.599000,0.011200,0.1830,0.261,130.969,"The lights go out, and I can't be saved\n Tide..."
3,0BCPKOYdS2jbQ8iyB56Zns,Clocks,Coldplay,f3e946cfb7048b4c7a23240f4d8c94273f01f68a,1,307879,0.577,0.749,5.0,-7.215,0.0,0.0279,0.599000,0.011200,0.1830,0.261,130.969,"The lights go out, and I can't be saved\n Tide..."
4,0BCPKOYdS2jbQ8iyB56Zns,Clocks,Coldplay,ba1defc039363ce394cc08a5a3fba7b86df7a2d6,1,307879,0.577,0.749,5.0,-7.215,0.0,0.0279,0.599000,0.011200,0.1830,0.261,130.969,"The lights go out, and I can't be saved\n Tide..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302613,5lQatPubYHgBIK22beDTi4,Keyword,9mm Parabellum Bullet,85aa2a02ba31bf7d247f4b72585ac2a9cbeb97a5,2,203226,0.279,0.974,8,-3.481,1,0.1260,0.000811,0.000041,0.0477,0.256,192.071,閉ざされたドアを開けるのは\n 取って付けられた合言葉\n 一度だけしか言わないから\n 愛...
1302614,5lQatPubYHgBIK22beDTi4,Keyword,9mm Parabellum Bullet,6364120099dfc1346e45123d3b90978eaa9fa419,1,203226,0.279,0.974,8,-3.481,1,0.1260,0.000811,0.000041,0.0477,0.256,192.071,閉ざされたドアを開けるのは\n 取って付けられた合言葉\n 一度だけしか言わないから\n 愛...
1302615,5lQatPubYHgBIK22beDTi4,Keyword,9mm Parabellum Bullet,ae6817c823f0e9837d01660c04f9e6d061295055,1,203226,0.279,0.974,8,-3.481,1,0.1260,0.000811,0.000041,0.0477,0.256,192.071,閉ざされたドアを開けるのは\n 取って付けられた合言葉\n 一度だけしか言わないから\n 愛...
1302616,5lQatPubYHgBIK22beDTi4,Keyword,9mm Parabellum Bullet,29676d2cc9920f9b44484824c7276eb0fbdbf4fd,5,203226,0.279,0.974,8,-3.481,1,0.1260,0.000811,0.000041,0.0477,0.256,192.071,閉ざされたドアを開けるのは\n 取って付けられた合言葉\n 一度だけしか言わないから\n 愛...


<h3>데이터 전처리</h3>

1. 영어 가사 추출 

In [ ]:
DetectorFactory.seed = 0

# 영어 가사만 추출
def is_english(lyrics):
    try:
        return detect(lyrics) == 'en'
    except:
        return False  


user_tracks['is_english'] = user_tracks['lyrics'].swifter.apply(is_english)
english_lyrics_df = user_tracks[user_tracks['is_english']]

Pandas Apply: 100%|██████████| 1302618/1302618 [2:14:39<00:00, 161.23it/s] 


In [ ]:
DetectorFactory.seed = 0  

def is_clean_english(lyrics):
    try:
        # 2. Must be mostly ASCII (filters out Japanese, Chinese, etc.)
        non_ascii_ratio = sum(1 for c in lyrics if ord(c) > 127) / max(len(lyrics), 1)
        return non_ascii_ratio < 0.1  # Allow up to 10% non-ASCII chars
    except:
        return False

# 영어가사 -> is_english : true
english_lyrics_df['is_english'] = english_lyrics_df['lyrics'].swifter.apply(is_clean_english)
english_lyrics_df = english_lyrics_df[english_lyrics_df['is_english']]

Pandas Apply: 100%|██████████| 1242378/1242378 [2:07:23<00:00, 162.55it/s] 
C:\Users\kist0\AppData\Local\Temp\ipykernel_5816\1042072688.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  english_lyrics_df['is_english'] = english_lyrics_df['lyrics'].swifter.apply(is_clean_english)


In [23]:
english_lyrics_df['is_english'].unique()

array([ True])

In [ ]:
# 랜덤으로 샘플 뽑아서 영어 가사 확인

print(english_lyrics_df['lyrics'].sample(10, random_state=22).tolist())

["Got a lotta heart ache\n He's a fuckin' weasel\n His issues make my mind ache\n Wanna make a deal\n 'Cause I love your little motions\n You do with your pigtails\n What a nice creation\n Worth another night in jail\n ♪\n He's a player, diarrhea giver\n Tried to grow his hair out, friends were listening to slayer\n I'd like to find him friday night\n Hanging out with mom, trying on his father's tights\n Life just sucks, I lost the one\n I'm giving up, she found someone\n There's plenty more\n Girls are such a drag\n ♪\n So all you little ladies\n Be sure to choose the right guys\n You'll come back to me maybe\n I'll shower you with lies\n Got a lotta heart ache\n He's a fuckin' weasel\n Decisions make my mind ache\n Wanna make a deal\n ♪\n Ease away the problems and the pain\n The girl chose the guy that makes you wanna kick and scream\n All along, you wish that she would stay\n Fuck the guy that took and ran away\n He's a player, diarrhea giver\n Tried to grow his hair out, friends w

In [30]:
english_lyrics_df = pd.read_csv("eng_user_lyrics.csv" , index_col=0)

C:\Users\kist0\AppData\Local\Temp\ipykernel_9944\2517621408.py:1: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  english_lyrics_df = pd.read_csv("eng_user_lyrics.csv" , index_col=0)


In [31]:
english_lyrics_df.head()

,track_uri,track_name,artist,user_id,playcount,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,lyrics,is_english
0,0BCPKOYdS2jbQ8iyB56Zns,Clocks,Coldplay,7241c0f8e15a4b9792ce5ddcfbe87a888f7f4b16,2,307879,0.577,0.749,5.0,-7.215,0.0,0.0279,0.599,0.0112,0.183,0.261,130.969,"The lights go out, and I can't be saved\n Tide...",True
1,0BCPKOYdS2jbQ8iyB56Zns,Clocks,Coldplay,8b5838c03d615893231d98995ff57d3f7a6659b9,4,307879,0.577,0.749,5.0,-7.215,0.0,0.0279,0.599,0.0112,0.183,0.261,130.969,"The lights go out, and I can't be saved\n Tide...",True
2,0BCPKOYdS2jbQ8iyB56Zns,Clocks,Coldplay,0fadffbb32e66100e27afc6a8a4a758ad8e9278b,1,307879,0.577,0.749,5.0,-7.215,0.0,0.0279,0.599,0.0112,0.183,0.261,130.969,"The lights go out, and I can't be saved\n Tide...",True
3,0BCPKOYdS2jbQ8iyB56Zns,Clocks,Coldplay,f3e946cfb7048b4c7a23240f4d8c94273f01f68a,1,307879,0.577,0.749,5.0,-7.215,0.0,0.0279,0.599,0.0112,0.183,0.261,130.969,"The lights go out, and I can't be saved\n Tide...",True
4,0BCPKOYdS2jbQ8iyB56Zns,Clocks,Coldplay,ba1defc039363ce394cc08a5a3fba7b86df7a2d6,1,307879,0.577,0.749,5.0,-7.215,0.0,0.0279,0.599,0.0112,0.183,0.261,130.969,"The lights go out, and I can't be saved\n Tide...",True


2. playcount (재생횟수) 범위 나누기

In [32]:
print(min(english_lyrics_df['playcount'].unique()))
print(max(english_lyrics_df['playcount'].unique()))

1
1862


In [33]:
# 10번 이상 들은 음악 -> 좋아하는 음악이라고 두고, 10번 이상 : 1, 이하: 0 으로 분류

english_lyrics_df['like'] = english_lyrics_df['playcount'] >= 10 
english_lyrics_df['like'] = english_lyrics_df['like'].astype(int)


In [34]:
print(english_lyrics_df['like'].value_counts().sort_index())

like
0    1189403
1      52049
Name: count, dtype: int64


In [35]:
# 0과 1 개수 맞춰줌 (1 개수에 맞추어 0 샘플 뽑아냄)

majority = english_lyrics_df[english_lyrics_df['like'] == 0]
minority = english_lyrics_df[english_lyrics_df['like'] == 1]

majority_downsampled = resample(majority, replace=False, n_samples=len(minority), random_state=42)
balanced_df = pd.concat([majority_downsampled, minority])

In [36]:
print(balanced_df['like'].value_counts().sort_index())

like
0    52049
1    52049
Name: count, dtype: int64


In [37]:
balanced_df.head()

,track_uri,track_name,artist,user_id,playcount,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,lyrics,is_english,like
1107812,0qCQg5TkfBfkTsQP3IhAmC,Southside Of Heaven,Ryan Bingham,95797a27c7ec61a04f0c9437448eee7233e8a456,1,379160,0.387,0.499,7.0,-10.207,1.0,0.0259,0.002900,0.003730,0.0695,0.333,111.032,"When I die, Lord, oh, won't you put my soul up...",True,0
702754,5B3UjDMiCwWFgnXqt0isd5,Martin Sheen Or JFK,Yellowcard,2e91726a1f802c680a8499544edc3561ce50b965,3,226746,0.256,0.756,2,-5.395,1,0.0390,0.000128,0.000000,0.1160,0.268,197.512,Here it is\n One more glass for these broken h...,True,0
394862,23qnota5Iyg4f9LNdNIC1D,Televators,The Mars Volta,aafd6caf4b6dad692dd93623cc5c4e5b5a919600,3,378786,0.306,0.541,11.0,-8.195,0.0,0.0383,0.457000,0.000006,0.1220,0.183,122.041,Just as he hit\n The ground\n They lowered a t...,True,0
213666,2sy0icOIskeP2lCqgZiTyE,Talk Show Host,Radiohead,28b78e314a00f86e24ae7ab1e40392861fbdc5a1,1,281000,0.535,0.479,5,-14.112,0,0.0311,0.277000,0.028500,0.1390,0.504,88.841,I want to\n I want to be someone else or I'll ...,True,0
1145411,19olraJp56OeGTQSZwLZc1,Dying Inside,Saint Vitus,8c5d4fe93b5eebbbbe5b80fdac548920c08c395c,1,445126,0.313,0.406,1.0,-14.754,1.0,0.0347,0.027800,0.000437,0.1250,0.218,112.856,I have got to change my ways\n 'Cause I'm losi...,True,0


In [38]:
balanced_df = balanced_df.reset_index(drop=True)
balanced_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104098 entries, 0 to 104097
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   track_uri         104098 non-null  object 
 1   track_name        104098 non-null  object 
 2   artist            104098 non-null  object 
 3   user_id           104098 non-null  object 
 4   playcount         104098 non-null  int64  
 5   duration_ms       104098 non-null  int64  
 6   danceability      104098 non-null  float64
 7   energy            104098 non-null  float64
 8   key               104098 non-null  object 
 9   loudness          104098 non-null  float64
 10  mode              104098 non-null  object 
 11  speechiness       104098 non-null  float64
 12  acousticness      104098 non-null  float64
 13  instrumentalness  104098 non-null  float64
 14  liveness          104098 non-null  float64
 15  valence           104098 non-null  float64
 16  tempo             10

In [41]:
balanced_df = balanced_df.drop(columns=['playcount', 'is_english']).reset_index(drop=True)

In [42]:
balanced_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104098 entries, 0 to 104097
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   track_uri         104098 non-null  object 
 1   track_name        104098 non-null  object 
 2   artist            104098 non-null  object 
 3   user_id           104098 non-null  object 
 4   duration_ms       104098 non-null  int64  
 5   danceability      104098 non-null  float64
 6   energy            104098 non-null  float64
 7   key               104098 non-null  object 
 8   loudness          104098 non-null  float64
 9   mode              104098 non-null  object 
 10  speechiness       104098 non-null  float64
 11  acousticness      104098 non-null  float64
 12  instrumentalness  104098 non-null  float64
 13  liveness          104098 non-null  float64
 14  valence           104098 non-null  float64
 15  tempo             104098 non-null  float64
 16  lyrics            10

3. key, mode 수치화

In [44]:
print(balanced_df['key'].unique())
print(balanced_df['mode'].unique())

['7.0' '2' '11.0' '5' '1.0' '6' '0.0' '5.0' 1 '4' '3' 9.0 '7' '8' '9'
 '9.0' '4.0' 11.0 '10' '11' 2.0 '6.0' 'E' 5 '0' '1' 8 '10.0' 'G#' '2.0'
 7.0 'D' 'F#' 'G' 'C#' '8.0' 'B' 'C' 'A#' 0.0 '3.0' 10.0 'F' 'A' 6.0 3.0
 'D#' 4.0]
['1.0' '1' '0.0' '0' 1 0.0 'Minor' 'Major']


key : E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1.  
mode : Major is represented by 1 and minor is 0. 

In [ ]:
note_to_int = {
    'C': 0, 'C#': 1, 'D': 2, 'D#': 3, 'E': 4, 'F': 5, 'F#': 6,
    'G': 7, 'G#': 8, 'A': 9, 'A#': 10, 'B': 11
}

def convert_key(val):
    try:
        return int(float(val))  # int로 변경
    except:
        return note_to_int.get(str(val), -1)  # -1 for unknowns

balanced_df['key'] = balanced_df['key'].apply(convert_key)


In [48]:
def convert_mode(val):
    str_val = str(val).strip().lower()
    if str_val in ['1', '1.0', 'major']:
        return 1
    elif str_val in ['0', '0.0', 'minor']:
        return 0
    else:
        return -1  

balanced_df['mode'] = balanced_df['mode'].apply(convert_mode)


In [49]:
print(balanced_df['key'].unique())
print(balanced_df['mode'].unique())

[ 7  2 11  5  1  6  0  4  3  9  8 10]
[1 0]


In [50]:
balanced_df.to_csv("balanced_lyrics.csv")